In [1]:
import requests
import pandas as pd
import logging
import time
import subprocess
import re
import traceback
from subprocess import STDOUT, check_output
from multiprocessing import Process, Value, Lock

In [2]:
def ping(ip, count=100):
    cmd = ["ping", ip, "-c{}".format(count), "-i0.1"]
    out = check_output(cmd, stderr=STDOUT, timeout=count + 60)
    out = out.decode("utf8").strip('\n')
    lines = out.split('\n')
    stat_dict = dict()
    m = re.search("(\d+) packets transmitted, (\d+) packets received", lines[-2])
    trans, rev = m.groups()
    fields = list(lines[-1].strip().split(" "))
    equal_index = fields.index("=")
    names = fields[equal_index - 1].split('/')
    values = [float(x) for x in fields[equal_index + 1].split('/')]
    kvs = zip(names, values)
    stat_dict = {
        "transmitted": int(trans),
        "received": int(rev),
        "loss_rate": (float(trans) - float(trans)) / float(trans)
    }
    for k, v in kvs:
        stat_dict[k] = v
    return stat_dict

In [3]:
class Counter(object):
    def __init__(self, initval=0):
        self.val = Value('i', initval)
        self.lock = Lock()

    def increase(self, incr):
        with self.lock:
            self.val.value += incr

    def value(self):
        with self.lock:
            return self.val.value

def download(file_url, counter):
    r = requests.get(file_url, stream=True)
    chunk_size = 1024
    for chunk in r.iter_content(chunk_size=chunk_size):
        counter.increase(chunk_size)

In [4]:
def getBandwith(file_url, process_cnt=20, duration=10):
    start_time = time.time()
    end_time = -1
    counter = Counter(0)
    procs = [Process(target=download, args=(file_url, counter)) for i in range(process_cnt)]
    for p in procs:
        p.start()
    while True:
        secs = time.time() - start_time
        if secs > duration:
            for p in procs:
                if p.is_alive():
                    p.kill()
            break
        time.sleep(1)
    counter.lock.acquire(timeout=0.5)
    counter.lock.release()
    total_size = counter.value() / 1024 / 1024
    bandwith = total_size / secs;
    # print("{}M / {:.02f}s = {:.02f}Mbps".format(total_size, secs, bandwith))
    return bandwith

In [5]:
def formatValue(value):
    if type(value) is str:
        return '"{}"'.format(value)
    if type(value) is float:
        return '{:.2f}'.format(value)
    return str(value)

In [6]:
df = pd.read_csv("./vultr.csv")
output_path = "./vultr_statistics.csv"
output_obj = open(output_path, "w", buffering=1)
colnames = ["name", "loss_rate", "avg_rtt", "stddev_rtt", "bandwith"]
output_obj.write(",".join(colnames) + "\n")

for index, r in df.iterrows():
    # print("Testing {}".format(r["name"]))
    try:
        ping_stat = ping(r["addr"], 200)
        try:
            bandwith = getBandwith(r["test_file"], 20, 10)
        except Exception as e:
            print(traceback.format_exc())
            bandwith = -1
        output_dict = {
            "name": r["name"],
            "loss_rate": ping_stat["loss_rate"],
            "avg_rtt": ping_stat["avg"],
            "stddev_rtt": ping_stat["stddev"],
            "bandwith": bandwith
        }
        output_obj.write(",".join([formatValue(output_dict[k]) for k in colnames]) + "\n")
    except Exception as e:
        print("Failed")
        print(traceback.format_exc())
output_obj.close()
print("test finished")
res_df = pd.read_csv(output_path)

Testing Seattle, Washington
Testing Sydney, Australia
Testing Dallas, Texas
Testing Silicon Valley, California
Testing Atlanta, Georgia
Testing Toronto, Canada
Testing Frankfurt, DE
Testing Chicago, Illinois
Testing Amsterdam, NL
Testing Miami, Florida
Testing Los Angeles, California
Testing Tokyo, Japan
Testing Singapore
Testing London, UK
Testing Paris, France
Testing New York (NJ)
test finished
